# Часть 1 (анализ датасета)
### (Построение выводов будет находиться под определенными ячейками)
## Импортирование нужных библиотек и первый взгляд на данные

In [32]:
import pandas
import numpy
from langdetect import detect
pandas.set_option('display.max_columns', 70)
initfile = pandas.read_csv('data.csv')
initfile.head(3)

,title,summary,url,approval_date_iso8601,stream_name,approved_by_name,clicks_fb,shares_fb,click_rate_fb,engagement_rate_fb,engagements_fb,clicks_in,shares_in,click_rate_in,engagement_rate_in,engagements_in,clicks_tw,shares_tw,click_rate_tw,engagement_rate_tw,engagements_tw,clicks_xi,shares_xi,click_rate_xi,engagement_rate_xi,engagements_xi,clicks_email,shares_email,click_rate_email,clicks_gp,shares_gp,click_rate_gp,id,suggested_by_name,image_url,clicks_total,engagements_total,click_rate_avg,engagement_rate_avg,shares_total,sharer_count,group_id,group_count,enterprise_group_id,sharing_rate,Article Tags
0,"Друзья, поздравляем вас с Новым Годом! https:/...","Друзья, поздравляем вас с Новым Годом! https:/...",https://twitter.com/Kaspersky_ru/status/121226...,2020-01-01T08:52:48Z,RU - Company Stream,NaN,0,1,0.0,0.0,0,0,3,0.0,0.0,0,11,5,2.2,0.0,0,2,1,2.0,0.0,0,0,0,0,0,0,0,9079866,NaN,NaN,13,0,1.3,0.0,10,6,134619,123,134619,0.0488,NaN
1,Social Media Detox: 5 Gründe für eine Pause vo...,Social Media kann sich ernsthaft auf die menta...,https://www.elle.de/social-media-detox,2020-01-02T07:45:09Z,DE - Company Stream,Michael Roesner,0,0,0.0,0.0,0,0,2,0.0,0.0,0,1,1,1.0,1.0,1,2,1,2.0,0.0,0,0,0,0,0,0,0,9084695,NaN,https://www.elle.de/sites/default/files/styles...,3,1,0.8,0.3,4,2,135166,53,135166,0.0377,NaN
2,Ältere mit Technik-Geschenken oft überfordert,Weihnachten ist vorbei und bestimmt gab es wie...,https://www.verivox.de/nachrichten/aeltere-mit...,2020-01-02T07:48:28Z,DE - Company Stream,Michael Roesner,0,0,0.0,0.0,0,0,0,0.0,0.0,0,1,1,1.0,1.0,1,2,1,2.0,0.0,0,0,0,0,0,0,0,9084696,NaN,NaN,3,1,1.5,0.5,2,1,135166,53,135166,0.0189,NaN


## Анализ представленных данных

### Просмотрев все признаки могу сказать, что нам, судя по всему, представлены данные об активности пользователей с разных социальных сетей, которые просматривают сообщения компаний, предоставляющих услуги распространения контента (это также могут быть официальные сайты, например официальный сайт Касперского).

### Сразу заметно большое количество столбцов, однако, насколько я могу предположить, они связаны по 5 штук: окончания наших признаков очень похожи на сокращения наименований соцсетей (tw - twitter, fb - facebook, in - instagram, xi - Xiaohongshu, gp - Google+). Таким образом у нас есть данные о количестве кликов на сообщение от каждой из соцсетей, количество поделившихся записью, коэффициент вовлеченности  и общая вовлеченность. 

#### P.S: Из чего рассчитывается коэффициент вовлеченности для разных сетей нам не дано, однако я не думаю, что с этим параметром удастся что-то сделать; для активности пользователей хватит количества людей, поделившихся сообщением и количества кликов. Впрочем, возможно, что вовлеченность из них и рассчитывается. 
#### Как определяются клики тоже неясно, однако я осмелюсь предположить, что это либо переход на ссылку из соцсети или пользователь, залогинившись, просматривает запись.

In [16]:
initfile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10486 entries, 0 to 10485
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  10469 non-null  object 
 1   summary                10331 non-null  object 
 2   url                    10467 non-null  object 
 3   approval_date_iso8601  10486 non-null  object 
 4   stream_name            10486 non-null  object 
 5   approved_by_name       3949 non-null   object 
 6   clicks_fb              10486 non-null  int64  
 7   shares_fb              10486 non-null  int64  
 8   click_rate_fb          10486 non-null  float64
 9   engagement_rate_fb     10486 non-null  float64
 10  engagements_fb         10486 non-null  int64  
 11  clicks_in              10486 non-null  int64  
 12  shares_in              10486 non-null  int64  
 13  click_rate_in          10486 non-null  float64
 14  engagement_rate_in     10486 non-null  float64
 15  en

#### Как можно заметить, у нас есть несколько признаков, столбцы которых имеют очень большое количество пропусков: Article Tags, suggested_by_name, approved_by_name. Я не думаю, что подобные признаки особо влияют на популярность поста, однако стоит проверить это позже, возможно есть корреляция между популярностью записи и этими признаками.

In [21]:
initfile.describe()

,clicks_fb,shares_fb,click_rate_fb,engagement_rate_fb,engagements_fb,clicks_in,shares_in,click_rate_in,engagement_rate_in,engagements_in,clicks_tw,shares_tw,click_rate_tw,engagement_rate_tw,engagements_tw,clicks_xi,shares_xi,click_rate_xi,engagement_rate_xi,engagements_xi,clicks_email,shares_email,click_rate_email,clicks_gp,shares_gp,click_rate_gp,id,clicks_total,engagements_total,click_rate_avg,engagement_rate_avg,shares_total,sharer_count,group_id,group_count,enterprise_group_id,sharing_rate
count,10486.000000,10486.000000,10486.000000,10486.0,10486.0,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.0,10486.0,10486.000000,10486.000000,10486.000000,10486.0,10486.0,10486.0,1.048600e+04,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000,10486.000000
mean,0.758058,0.950792,0.441865,0.0,0.0,1.936391,4.156018,0.542953,0.724099,3.227255,15.323002,4.561892,2.846185,0.244669,1.140950,2.189395,1.043963,0.956714,0.0,0.0,0.003242,0.021076,0.003052,0.0,0.0,0.0,1.076456e+07,20.210090,4.368205,1.863694,0.512817,10.733931,4.415602,138624.364486,168.857524,138624.364486,0.044500
std,4.923595,1.205559,3.341166,0.0,0.0,5.511444,4.785178,2.074960,2.575696,7.603575,50.029692,5.161311,3.990378,0.611358,3.113245,4.630824,1.589808,1.937251,0.0,0.0,0.090508,0.174246,0.086197,0.0,0.0,0.0,9.610718e+05,52.673611,9.088802,2.669028,2.232113,11.769555,4.475726,23123.525910,154.497068,23123.525910,0.093833
min,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,9.079866e+06,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,128482.000000,5.000000,128482.000000,0.002400
25%,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,9.940828e+06,3.000000,0.000000,0.900000,0.000000,3.000000,1.000000,128490.000000,41.000000,128490.000000,0.017100
50%,0.000000,1.000000,0.000000,0.0,0.0,0.000000,2.000000,0.000000,0.000000,0.000000,6.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.076363e+07,9.000000,1.000000,1.300000,0.200000,6.000000,3.000000,134619.000000,111.000000,134619.000000,0.027800
75%,0.000000,2.000000,0.000000,0.0,0.0,2.000000,5.000000,0.500000,1.000000,4.000000,15.000000,6.000000,3.500000,0.200000,1.000000,2.000000,2.000000,2.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.158732e+07,22.000000,5.000000,2.100000,0.500000,14.000000,5.000000,137471.000000,343.000000,137471.000000,0.048800
max,149.000000,11.000000,149.000000,0.0,0.0,267.000000,115.000000,75.000000,197.000000,254.000000,3977.000000,118.000000,192.000000,14.000000,107.000000,217.000000,34.000000,72.300000,0.0,0.0,4.000000,7.000000,4.000000,0.0,0.0,0.0,1.245892e+07,4008.000000,258.000000,91.000000,197.000000,269.000000,36.000000,281456.000000,474.000000,281456.000000,6.600000


#### Глядя на распределение можно сказать, что, в общем, наши представленные сообщения очень непопулярны: хоть сколько-нибудь значимые значения идут от 3 квартиля. Но это можно считать плюсом, потому что вкупе со значением max, все это нам говорит о большом разбросе в данных, т.е. где-то есть очень популярные сообщения. 

#### Пока остановимся здесь и распределим наши соцсети по популярности: 
1. tw
2. in
3. xi
4. Email
5. gp
#### Это небольшая прихоть, но все-таки я бы проверил максимально популярные посты по китайской соцсети, потому что про Xiaohongshu я никогда не слышал, и в рейтингах сеть отнюдь не самая поплуряная в Китае. Может само сообщение этому как-то способствует, проверим:

#### Для начала посмотрим на stream_name, в котором должен храниться язык компании, предоставляющей услуги продвижения контента

In [31]:
initfile['stream_name'].unique()

array(['RU - Company Stream', 'DE - Company Stream',
       'NA - Company Stream', 'EN - B2B', 'EN - B2C',
       'LATAM - Company Stream', 'EN - Our KL ambassadors',
       'EN - Company Stream', 'ME - Company Stream', 'UK - B2C',
       'RU - B2B', 'RU - B2C', 'RU - Eugene', 'BR - Company Stream',
       'IT - Company Stream', 'TR - Company Stream',
       'JP - Company Stream', 'EN - Industry news',
       'IN  - Company Stream', 'RU - HR (vacancies, events, news)',
       'APAC - Must Read & Share', 'ESP - Company Stream',
       'FR - Company Stream', 'NA - B2C', 'EN - Work from Home',
       'Must read and share!', 'EN - Women In CyberSec',
       'BR - Enterprise Sales', 'DACH - Enterpise Sales',
       'PT - Company Stream', 'RU - Fan Club News', 'Ferrari Contest',
       'RU - Must read and share', 'UK - B2B'], dtype=object)

#### В stream_name китайской компании не оказалось, однако это не значит, что сообщений на китайском не было, попробуем достать их из title.

In [36]:
languages = set()
for title in initfile['title']:
    languages.add(detect(str(title)))
languages

{'af',
 'ar',
 'bg',
 'ca',
 'cs',
 'cy',
 'da',
 'de',
 'en',
 'es',
 'et',
 'fi',
 'fr',
 'hr',
 'id',
 'it',
 'ja',
 'lt',
 'mk',
 'nl',
 'no',
 'pl',
 'pt',
 'ro',
 'ru',
 'sk',
 'sl',
 'so',
 'sq',
 'sv',
 'sw',
 'tl',
 'tr',
 'uk',
 'zh-cn'}

#### Все-таки китайский есть: "zh-cn", пожалуй стоит в датафрейм добавить столбец с языком, чтобы было легче найти эти сообщения, заодно посмотрим на распределение сообщений по языкам

In [51]:
initfile['language']=initfile['title'].apply(lambda title: detect(str(title)))

In [58]:
initfile[initfile['language'] == 'zh-cn'].head()

,title,summary,url,approval_date_iso8601,stream_name,approved_by_name,clicks_fb,shares_fb,click_rate_fb,engagement_rate_fb,engagements_fb,clicks_in,shares_in,click_rate_in,engagement_rate_in,engagements_in,clicks_tw,shares_tw,click_rate_tw,engagement_rate_tw,engagements_tw,clicks_xi,shares_xi,click_rate_xi,engagement_rate_xi,engagements_xi,clicks_email,shares_email,click_rate_email,clicks_gp,shares_gp,click_rate_gp,id,suggested_by_name,image_url,clicks_total,engagements_total,click_rate_avg,engagement_rate_avg,shares_total,sharer_count,group_id,group_count,enterprise_group_id,sharing_rate,Article Tags,language
4681,电子游戏安全省钱大法,盗版游戏、灰色市场激活码和官方商城现成账号的危害...,https://www.kaspersky.com.cn/blog/whats-wrong-...,2020-06-05T08:01:49Z,APAC - Must Read & Share,Sanjeev,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,1,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,10555087,NaN,https://d1tfz9268y8c5s.cloudfront.net/curated_...,0,0,0.0,0.0,1,1,151740,91,151740,0.011,NaN,zh-cn


#### К сожалению, теория не оправдалась - у нас лишь одно сообщение на китайском. Тем более, что единственная активность, связанная с этим сообщением - одно событие "поделиться" из твиттера.

#### Посмотрим на распределение по языкам


In [76]:
initfile['language'].value_counts()

en       4894
ru       1917
es       1141
de        546
pt        386
tr        301
ca        260
it        215
ar        150
fr        132
bg        131
af         64
sv         45
no         35
tl         33
ja         32
mk         31
ro         22
da         22
id         21
sk         18
nl         17
uk         14
et         13
sw         10
fi          8
cs          7
lt          4
so          4
hr          4
sl          4
cy          2
sq          1
pl          1
zh-cn       1
Name: language, dtype: int64

#### Как можно заметить, самым используемым для сообщений языком является английский, почетные второе и третье места занимают русский и испанский. Конечно, нам это не говорит о том, что и популярность сообщений будет распределяться также, но наверняка она будет чуть ли не такой же.



In [79]:
pandas.crosstab(initfile['language'], max(initfile['click_rate_avg']))

col_0,91.0
language,
af,64
ar,150
bg,131
ca,260
cs,7
cy,2
da,22
de,546
en,4894


#### Конечно недостаточно посмотреть только на максимум, но я думаю, что ничего другого мы не увидим - распределения слишком похожи.

In [10]:
initfile['Article Tags'].unique()

array([nan, 'Important NA NF', 'Important', 'Ferrari contest'],
      dtype=object)

#### Хоть признак Article Tags имеет большое количество пропусков, однако в нем есть значение "Important", скорее всего популярность таких сообщений должна быть выше, чем у остальных:

In [122]:
pandas.crosstab(initfile['Article Tags'], sum(initfile['click_rate_avg']))

col_0,19542.7
Article Tags,
Ferrari contest,10
Important,10
Important NA NF,1070


#### Как оказалось, у тэга important NA NF больше всего активности. Но если мы посмотрим на обычную сортировку по кликам, то увидимо, что самые популярные сообщения могут быть элементарно не отмечены тэгом:

In [77]:
initfile.sort_values('click_rate_avg', ascending=False)

,title,summary,url,approval_date_iso8601,stream_name,approved_by_name,clicks_fb,shares_fb,click_rate_fb,engagement_rate_fb,engagements_fb,clicks_in,shares_in,click_rate_in,engagement_rate_in,engagements_in,clicks_tw,shares_tw,click_rate_tw,engagement_rate_tw,engagements_tw,clicks_xi,shares_xi,click_rate_xi,engagement_rate_xi,engagements_xi,clicks_email,shares_email,click_rate_email,clicks_gp,shares_gp,click_rate_gp,id,suggested_by_name,image_url,clicks_total,engagements_total,click_rate_avg,engagement_rate_avg,shares_total,sharer_count,group_id,group_count,enterprise_group_id,sharing_rate,Article Tags,language
3472,"Security Analyst Summit (SAS) – April 28-30, 2...",The Kaspersky Security Analyst Summit (SAS) is...,https://thesascon.com/SAS@home?utm_source=twit...,2020-04-27T19:17:58Z,NA - Company Stream,Zoe Tan,0,0,0.0,0.0,0,0,0,0.0,0.0,0,91,1,91.0,6.0,6,0,0,0.0,0.0,0,0,0,0,0,0,0,10175742,NaN,https://d1tfz9268y8c5s.cloudfront.net/curated_...,91,6,91.0,6.0,1,1,134890,72,134890,0.0139,NaN,en
3565,Four-in-Five People Have Tried to Remove Priva...,New Kaspersky research has found that consumer...,https://usa.kaspersky.com/about/press-releases...,2020-04-29T15:30:19Z,NA - Company Stream,Zoe Tan,0,0,0.0,0.0,0,0,0,0.0,0.0,0,84,1,84.0,6.0,6,0,0,0.0,0.0,0,0,0,0,0,0,0,10195868,NaN,https://d1tfz9268y8c5s.cloudfront.net/curated_...,84,6,84.0,6.0,1,1,134890,72,134890,0.0139,Important NA NF,en
8461,Kaspersky tem novo responsável de parceiros em...,Élio de Oliveira é o novo Territory Channel ma...,https://executivedigest.sapo.pt/kaspersky-tem-...,2020-10-13T12:48:15Z,PT - Company Stream,Alberto,0,0,0.0,0.0,0,75,1,75.0,197.0,197,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,11788178,NaN,https://d1tfz9268y8c5s.cloudfront.net/curated_...,75,197,75.0,197.0,1,1,263963,12,263963,0.0833,NaN,pt
2001,İnternet çerezleri siber saldırı amacıyla kull...,"Siber güvenlik uzmanları, tarayıcıdaki ve popü...",https://www.hurriyet.com.tr/teknoloji/internet...,2020-03-16T16:00:54Z,TR - Company Stream,Mücahit Hoscan,0,0,0.0,0.0,0,0,2,0.0,0.0,0,192,1,192.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,9767255,NaN,https://d1tfz9268y8c5s.cloudfront.net/curated_...,192,0,64.0,0.0,3,2,181067,8,181067,0.2500,NaN,tr
4031,"Transatlantic Cable podcast, episode 142","On this podcast, Dave and Jeff talk with Rob C...",https://www.kaspersky.com/blog/transatlantic-c...,2020-05-14T12:52:19Z,NA - Company Stream,NaN,0,0,0.0,0.0,0,0,0,0.0,0.0,0,61,1,61.0,11.0,11,0,0,0.0,0.0,0,0,0,0,0,0,0,10342467,NaN,https://www.kaspersky.com/blog/wp-content/plug...,61,11,61.0,11.0,1,1,134890,72,134890,0.0139,NaN,ca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7755,24 дня приобщения к аltaiской силище.,"Только что обнаружил, что по-английски Алтай п...",http://kas.pr/6dar,2020-09-17T08:52:18Z,EN - Our KL ambassadors,NaN,0,0,0.0,0.0,0,0,1,0.0,0.0,0,0,2,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,11551545,NaN,https://pbs.twimg.com/media/EiGHoHcWsAIUFBz.jpg,0,0,0.0,0.0,3,2,128483,401,128483,0.0050,NaN,bg
8369,Cyberwaffen und der Rattenfänger von Hameln,Die Legende um den Rattenfänger von Hameln wur...,https://www.kaspersky.de/blog/pied-piper-hamel...,2020-10-08T12:53:16Z,DE - Company Stream,NaN,0,0,0.0,0.0,0,0,1,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,11749372,NaN,https://media.kasperskydaily.com/wp-content/up...,0,0,0.0,0.0,1,1,135166,53,135166,0.0189,NaN,de
7510,O uso de pixel de rastreamento em cibercrimes,Cibercriminosos adotaram uma ferramenta de mar...,https://www.kaspersky.com.br/blog/tracking-pix...,2020-09-09T22:53:26Z,BR - Company Stream,NaN,0,0,0.0,0.0,0,0,1,0.0,4.0,4,0,2,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,11478132,NaN,https://media.kasperskydaily.com/wp-content/up...,0,4,0.0,1.3,3,2,137473,31,137473,0.0645,NaN,pt
10088,Kaspersky、2021年の高度サイバー攻撃に関する予測を発表,Kasperskyのグローバル調査分析チーム（GReAT）は、年次のサイバー脅威動向レポート...,https://enterprisezine.jp/news/detail/13701,2020-12-11T08

In [127]:
initfile[initfile['language'] == 'en']['title'].head(10)

6     Municipality Ransomware Attacks Up 60% in 2019...
7     Municipality Ransomware Attacks Up 60% in 2019...
8       7 security incidents that cost CISOs their jobs
9       7 security incidents that cost CISOs their jobs
10    Crooks use Star Wars saga as bait in Phishing ...
11    Crooks use Star Wars saga as bait in Phishing ...
12    Kaspersky protege de las amenazas web con Kasp...
21    Kaspersky: SMBs 'More Actively' Review IT Secu...
22    US Coast Guard Issues Warning Over Crypto Rans...
23    US Coast Guard Issues Warning Over Crypto Rans...
Name: title, dtype: object

#### Как можно заметить, некоторые записи повторяются, скорее всего, они просто представлены на разных ресурсах

In [129]:
  initfile[initfile['title'] == '7 security incidents that cost CISOs their jobs']

,title,summary,url,approval_date_iso8601,stream_name,approved_by_name,clicks_fb,shares_fb,click_rate_fb,engagement_rate_fb,engagements_fb,clicks_in,shares_in,click_rate_in,engagement_rate_in,engagements_in,clicks_tw,shares_tw,click_rate_tw,engagement_rate_tw,engagements_tw,clicks_xi,shares_xi,click_rate_xi,engagement_rate_xi,engagements_xi,clicks_email,shares_email,click_rate_email,clicks_gp,shares_gp,click_rate_gp,id,suggested_by_name,image_url,clicks_total,engagements_total,click_rate_avg,engagement_rate_avg,shares_total,sharer_count,group_id,group_count,enterprise_group_id,sharing_rate,Article Tags,language
8,7 security incidents that cost CISOs their jobs,Whether or not security executives lose their ...,https://www.csoonline.com/article/3510640/7-se...,2020-01-02T15:28:27Z,NA - Company Stream,Zoe Tan,0,0,0.0,0.0,0,0,0,0.0,0.0,0,7,1,7.0,2.0,2,0,0,0.0,0.0,0,0,0,0,0,0,0,9087508,NaN,https://images.idgesg.net/images/article/2019/...,7,2,7.0,2.0,1,1,134890,72,134890,0.0139,Important NA NF,en
9,7 security incidents that cost CISOs their jobs,Whether or not security executives lose their ...,https://www.csoonline.com/article/3510640/7-se...,2020-01-02T15:28:32Z,EN - B2B,Zoe Tan,1,7,0.1,0.0,0,2,20,0.1,0.4,7,129,23,5.6,0.7,17,10,5,2.0,0.0,0,0,0,0,0,0,0,9087509,NaN,https://images.idgesg.net/images/article/2019/...,142,24,2.6,0.4,55,24,128487,409,128487,0.0587,Important NA NF,en


#### Действительно так, сообщения представлены на разных ресурсах.

#### Я хотел бы также рассказать про теорию, которую не смогу показать наглядно, но у меня есть подозрения, что не только наши сообщения представлены на разных ресурсах, но и на разных языках, однако я не знаю, как это сделать.
#### Еще одна вещь: я бы хотел распределить с помощью признака summary и pyLDAvis наши строки по темам, чтобы найти самые популярные из них хотя бы на английском языке, но, к сожалению, у меня не хватает на это либо достаточных знаний для работы с этой библиотекой, либо вычислительных мощностей.

## Часть 2 (построение выводов)

#### Итак, мы имеем выборку, в которой представлены данные по распределению активности пользователей 5-ти социальных сетей относительно избранных сообщений. Большая половина сообщений представлена на английском языке, она же является самой популярной среди пользователей. Самые активные пользователи приходят из twitter. Больше 75% нашей выборки имеют малые параметры активности, следовательно, большая часть сообщений неинтересна пользователям. Наши сообщения представлены на нескольких ресурсах, причем активность пользователей от разных платформ распределяется примерно так же, как распределены по популярности социальные сети.

## Часть 3

#### Блогпост написан о способах взлома банковской карты и как себя от них защитить. Текст будет полезен для каждого обладателя банковской карты, а особенно для тех, кто не задумывается о ее безопасности, надеясь на защиту банка. Информация, как мне кажется, хорошо может быть репрезентована в анимированных картинках, в которых наглядно будет показана либо система безопасности (двухфакторная аутентификация или 3-d secure), либо события, в которых мошенники могут выкрасть карту/информацию с нее. Очень хорошо смотрелось бы на экранах вагонов метро.